In [0]:
############### Creating Variables ###########################
catalog = "flight"
source_schema = "silver"
source_object = "silver_booking"
target_schema = "gold"
target_object = "FactBookings"
cdc_col = "modifiedDate"
backdated_refresh = ""
fact_table = f"{catalog}.{source_schema}.{source_object}"
fact_key_col = ["DimFlightsKey", "DimPassengersKey", "DimAirportsKey","booking_date"]

Creating list of Diamentions

In [0]:
diamentions = [
  {
    "table" : "flight.gold.DimFlights",
    "alias" : "dimFlights",
    "join_keys" : [("flight_id", "flight_id")],
  },
  {
    "table" : "flight.gold.DimPassengers",
    "alias" : "dimPassengers",
    "join_keys" : [("passenger_id", "passenger_id")],
  },
  {
    "table" : "flight.gold.DimAirports",
    "alias" : "dimAirports",
    "join_keys" : [("airport_id", "airport_id")],
  }
]

fact_col = ["amount", "booking_date", "modifiedDate"]

Getting last_load date.

In [0]:
if len(backdated_refresh) == 0:
    if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):
        last_load = spark.sql(f"SELECT max({cdc_col}) FROM {catalog}.{target_schema}.{target_object}").collect()[0][0]
    else :
        last_load = "1900-01-01 00:00:00"
else:
    backdated_refresh = last_load

last_load

datetime.datetime(2025, 7, 10, 5, 52, 14, 168000)

### Dynamic Fact Query

In [0]:
def generate_fact_query_incremental(fact_table, dimensions, fact_columns, cdc_column, last_load):
    fact_alias = "f"
    
    # Base columns to select
    select_cols = [f"{fact_alias}.{col}" for col in fact_columns]

    # Build joins dynamically
    join_clauses = []
    for dim in dimensions:
        table_full = dim["table"]
        alias = dim["alias"]
        table_name = table_full.split('.')[-1]
        surrogate_key = f"{alias}.{table_name}Key"
        select_cols.append(surrogate_key)

        # Build ON clause
        on_conditions = [
            f"{fact_alias}.{fk} = {alias}.{dk}" for fk, dk in dim["join_keys"]
        ]
        join_clause = f"LEFT JOIN {table_full} {alias} ON " + " AND ".join(on_conditions)
        join_clauses.append(join_clause)

    # Final SELECT and JOIN clauses
    select_clause = ",\n    ".join(select_cols)
    joins = "\n".join(join_clauses)

    # WHERE clause for incremental filtering
    where_clause = f"{fact_alias}.{cdc_column} >= DATE('{last_load}')"

    # Final query
    query = f"""
SELECT
    {select_clause}
FROM {fact_table} {fact_alias}
{joins}
WHERE {where_clause}
""".strip()

    return query


In [0]:
query = generate_fact_query_incremental(fact_table, diamentions, fact_col, cdc_col, last_load)

In [0]:
query

"SELECT\n    f.amount,\n    f.booking_date,\n    f.modifiedDate,\n    dimFlights.DimFlightsKey,\n    dimPassengers.DimPassengersKey,\n    dimAirports.DimAirportsKey\nFROM flight.silver.silver_booking f\nLEFT JOIN flight.gold.DimFlights dimFlights ON f.flight_id = dimFlights.flight_id\nLEFT JOIN flight.gold.DimPassengers dimPassengers ON f.passenger_id = dimPassengers.passenger_id\nLEFT JOIN flight.gold.DimAirports dimAirports ON f.airport_id = dimAirports.airport_id\nWHERE f.modifiedDate >= DATE('2025-07-10 05:52:14.168000')"

In [0]:
df_fact = spark.sql(query)

In [0]:
# display(df_fact)

### Upsert

In [0]:
# Fact Key Columns Merge Condition
fact_key_cols_str = " AND ".join([f"src.{col} = trg.{col}" for col in fact_key_col])
fact_key_cols_str

'src.DimFlightsKey = trg.DimFlightsKey AND src.DimPassengersKey = trg.DimPassengersKey AND src.DimAirportsKey = trg.DimAirportsKey AND src.booking_date = trg.booking_date'

In [0]:
from delta.tables import DeltaTable

if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):

    dlt_obj = DeltaTable.forName(spark, f"{catalog}.{target_schema}.{target_object}")
    dlt_obj.alias("trg").merge(df_fact.alias("src"), fact_key_cols_str)\
                        .whenMatchedUpdateAll(condition = f"src.{cdc_col} >= trg.{cdc_col}")\
                        .whenNotMatchedInsertAll()\
                        .execute()

else: 

    df_fact.write.format("delta")\
            .mode("append")\
            .saveAsTable(f"{catalog}.{target_schema}.{target_object}")


In [0]:
%sql
-- select * from flight.gold.factbookings

amount,booking_date,modifiedDate,DimFlightsKey,DimPassengersKey,DimAirportsKey
770.85,2025-06-23,2025-07-10T05:52:14.168Z,68,184,10
1236.07,2025-05-01,2025-07-10T05:52:14.168Z,68,55,46
790.47,2025-06-18,2025-07-10T05:52:14.168Z,68,11,42
1272.72,2025-04-30,2025-07-10T05:52:14.168Z,68,119,34
555.91,2025-04-08,2025-07-10T05:52:14.168Z,68,172,46
372.89,2025-06-01,2025-07-10T05:52:14.168Z,68,62,3
368.63,2025-04-20,2025-07-10T05:52:14.168Z,68,23,30
1445.34,2025-04-09,2025-07-10T05:52:14.168Z,68,56,39
152.12,2025-04-26,2025-07-10T05:52:14.168Z,68,110,14
596.62,2025-05-05,2025-07-10T05:52:14.168Z,68,116,48
